In [ ]:
import pennylane as qml
from pennylane import numpy as np
import gym

# Quantum device
dev = qml.device("default.qubit", wires=4)




In [50]:
#Defining the quantum circuit
def quantum_nn(weights, x):
    qml.AngleEmbedding(x, wires=range(4))
    qml.StronglyEntanglingLayers(weights, wires=range(4))
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # or however many actions
# Return the expectation value of the first qubit

In [51]:
#Define the QNN Function
@qml.qnode(dev)
def quantum_forward(weights,x):
    return quantum_nn(weights,x)

In [ ]:

# Environment
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [ ]:
# Hyperparameters
epsilon = 0.1
learning_rate = 0.1
gamma = 0.99
episodes = 500

[tensor(-0.18159053, requires_grad=True), tensor(0.06803147, requires_grad=True), tensor(0.56981349, requires_grad=True), tensor(-0.31957494, requires_grad=True)]


In [ ]:

# Initialize weights: (layers, wires, params)
weights = np.random.uniform(0, np.pi, (4, 4, 3))


In [55]:
#Hyperparameters & Initialization
epsilon = 0.1
learning_rate = 0.1
gamma = 0.99
episodes = 500
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
weights = np.random.uniform(0, np.pi, (4,4, 3))

In [ ]:
# Training loop
for episode in range(episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        # Epsilon-greedy action selection
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            q_values = quantum_forward(weights, state)
            action = int(np.argmax(q_values))


In [ ]:
# Loop for each step in the episode
while not done: 
    if np.random.rand() < epsilon:
        action = env.action_space.sample()
    else:
        q_values = quantum_forward(weights, state)
        action = np.argmax(q_values)


In [ ]:
# Q Learning update
next_state, reward, done, _, _ = env.step(action)
total_reward += reward
q_next = max([quantum_forward(weights, next_state) for _ in range(action_size)])
q_value = quantum_forward(weights, state)[action]
weights -= learning_rate * (reward + gamma * q_next - q_value)
state = next_state
